# model SB With sigmod and binray output
CNN + RNN (LSTM) 
sara 


#  -------------------------------------------------------------------------------------------

# imports

In [1]:
import tensorflow as tf
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras_tqdm import TQDMNotebookCallback
import numpy as np
from keras_tqdm import TQDMNotebookCallback
import nltk
import xml.etree.ElementTree as ET
import pandas as pd
import os
import string
from nltk.tokenize import TreebankWordTokenizer
from numpy.random import random_sample
import re
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from keras.layers import Embedding, Flatten,LSTM
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation,  Input, merge,Conv1D,MaxPooling1D,GlobalMaxPooling1D,Convolution1D
from keras import regularizers
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from keras.layers import Concatenate, concatenate
from keras import backend as K
from keras.layers import multiply
from keras.layers import merge
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
random_seed=1337

Using TensorFlow backend.


###  Define Callback functions to generate Measures

In [2]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


# Experiments to reproduce the results of Table 9 

### Load pre procssed Data

In [12]:
with open('../data/pickles/train_and_test_data_sentences_snp_2class.pickle', 'rb') as handle:
#with open('../../SNP-Disease/train_and_test_data_sentences_snp_2classWiki.pickle', 'rb') as handle:

    W_train = pickle.load(handle)
    d1_train = pickle.load(handle)
    d2_train = pickle.load(handle)
    Y_train = pickle.load(handle)
    Y_train = np.argmax(Y_train, axis=1)
    Tr_word_list = pickle.load(handle)
    
    W_test = pickle.load(handle)
    d1_test = pickle.load(handle)
    d2_test = pickle.load(handle)
    Y_test = pickle.load(handle)
    Y_test = np.argmax(Y_test, axis=1)
    Te_word_list = pickle.load(handle)
    
    
    word_vectors = pickle.load(handle)
    word_dict = pickle.load(handle)
    d1_dict = pickle.load(handle)
    d2_dict = pickle.load(handle)
    label_dict = pickle.load(handle)
    MAX_SEQUENCE_LENGTH = pickle.load(handle)
    
    

### Prepare Word Embedding Layer

In [13]:
EMBEDDING_DIM=word_vectors.shape[1]
embedding_matrix=word_vectors

def create_embedding_layer(l2_reg=0.1,use_pretrained=True,is_trainable=False):
    
    if use_pretrained:        
        return Embedding(len(word_dict) ,EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=is_trainable,embeddings_regularizer=regularizers.l2(l2_reg))
    else:    
        return Embedding(len(word_dict) ,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH)
            

### Create the Model

In [31]:
def build_model():    
    
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedding_layer=create_embedding_layer(use_pretrained=True,is_trainable=False)
    embedded_sequences = embedding_layer(sequence_input)

    x = Conv1D(256, 7, activation='relu')(embedded_sequences)
    x = MaxPooling1D(3)(x)
    x = Dropout(0.5)(x)

    x = Conv1D(128, 5, activation='relu')(x)
    x = MaxPooling1D(3)(x)
    x = Dropout(0.5)(x)
    
 
    
    conv_sequence=GlobalMaxPooling1D()(x)    #x = Flatten()(x)

    
    forward = LSTM(200,recurrent_dropout=0.05)(embedded_sequences)
    backward = LSTM(200, go_backwards=True,recurrent_dropout=0.05)(embedded_sequences)
    lstm_sequence = concatenate([forward,backward])
    merge = concatenate([conv_sequence,lstm_sequence])
    x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.05))(merge)
    x = Dropout(0.5)(x)
    preds = Dense(1, activation='sigmoid')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',f1])
    #model.summary()
    return model

In [32]:
model = build_model()
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 91)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 91, 200)      555000      input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 85, 256)      358656      embedding_5[0][0]                
__________________________________________________________________________________________________
max_pooling1d_9 (MaxPooling1D)  (None, 28, 256)      0           conv1d_9[0][0]                   
____________________________________________________________________________________________

### Run the Evaluation on the test dataset 

In [35]:
param='macro'
epochs =50
batch_size = 64
history=model.fit(W_train, Y_train,epochs=epochs,validation_data=(W_test,Y_test), batch_size=batch_size, verbose=1,callbacks=[TQDMNotebookCallback()])        
       




Train on 935 samples, validate on 365 samples


Training:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50


Epoch 0:   0%|          | 0/935 [00:00<?, ?it/s]

935/935 [==============================] - 36s 38ms/step - loss: 5009.4344 - mse: 0.0457 - val_loss: 5009.7904 - val_mse: 0.1811
Epoch 2/50


...

935/935 [==============================] - 37s 39ms/step - loss: 5009.0989 - mse: 0.0019 - val_loss: 5009.6061 - val_mse: 0.1463
Epoch 49/50


Epoch 48:   0%|          | 0/935 [00:00<?, ?it/s]

935/935 [==============================] - 37s 40ms/step - loss: 5009.0909 - mse: 9.3928e-04 - val_loss: 5009.7312 - val_mse: 0.1793
Epoch 50/50


Epoch 49:   0%|          | 0/935 [00:00<?, ?it/s]

935/935 [==============================] - 37s 39ms/step - loss: 5009.0920 - mse: 0.0017 - val_loss: 5009.7048 - val_mse: 0.1710


In [36]:
#Training & Validation accuracy
# visualizing  accuracy
train_loss=history.history['loss']
val_loss=history.history['val_loss']
train_acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
xc = range(epochs)
plt.figure(1, figsize=(7, 5))
plt.plot(xc, train_loss)
plt.plot(xc, val_loss)
plt.xlabel('Number of Epochs')
plt.ylabel('Loss Function')
plt.title('Training Loss Function Vs Validation Loss Function')
plt.grid(True)
plt.legend(['Train', 'validation'])
# print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.figure(2, figsize=(7, 5))
plt.plot(xc, train_acc)
plt.plot(xc, val_acc)
plt.xlabel('Number of Epochs')
plt.ylabel('Train accuracy')
plt.title('Training Accuracy Vs Validation Accuracy')
plt.grid(True)
plt.legend(['train', 'validation'], loc=4)
# print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.show()

KeyError: 'accuracy'

In [37]:
print('Running predictions...')
all_predictions, all_labels = [], []
labels =Y_test
y_pred = model.predict(W_test)
all_predictions.extend(y_pred.astype('int32'))
all_labels.extend(labels.astype('int32'))
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

correct_pred_count = (all_labels == all_predictions).sum()
test_acc = correct_pred_count / len(all_labels)
# show the the accuracy of testing data
print('We got %d of %d correct (or %.3f accuracy)' % (correct_pred_count, len(all_labels), test_acc))
print('Accuracy:', accuracy_score(y_true=all_labels, y_pred=all_predictions))
print(classification_report(y_true=all_labels, y_pred=all_predictions))

Running predictions...
We got 71150 of 365 correct (or 194.932 accuracy)
Accuracy: 0.536986301369863
              precision    recall  f1-score   support

           0       0.54      1.00      0.70       195
           1       1.00      0.01      0.01       170

    accuracy                           0.54       365
   macro avg       0.77      0.50      0.35       365
weighted avg       0.75      0.54      0.38       365

